In [61]:
import pandas as pd
import numpy as np

import matplotlib.style as style
style.use('seaborn-whitegrid')

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.applications import VGG16
from keras.optimizers import SGD
from keras.preprocessing import image as image_utils
from keras.utils import to_categorical
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings(action='ignore')

In [75]:
gestures = {'L_': 'L',
           'fi': 'Fist',
           'C_': 'C',
           'ok': 'Okay',
           'pe': 'Peace',
           'pa': 'Palm'
            }

gestures_map = {'C': 0,
               'Fist' : 1,
               'L': 2,
               'Okay': 3,
               'Palm': 4,
               'Peace': 5
                }

#### Drawing data

In [101]:
X_data = []
y_data = []
image_path = []
gesture = []
image_rgb = []

# directory_in_str = '/home/ubuntu/project_kojak/frames/silhouettes'
directory = os.fsencode('/home/ubuntu/project_kojak/frames/drawings/')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"): 
        path = os.path.join('/home/ubuntu/project_kojak/frames/drawings/', filename)
        gesture_name = gestures[filename[8:10]]
        
        gesture.append(gesture_name)
        y_data.append(gestures_map[gesture_name])
        image_path.append(path)

        
        img = Image.open(path).convert('L')
        img = img.resize((224, 224))
        arr = np.array(img)
        X_data.append(arr)
        
        img2rgb = image_utils.load_img(path=path, target_size=(224, 224))
        img2rgb = image_utils.img_to_array(img2rgb)
        image_rgb.append(img2rgb)

        


#         y_values = np.full((count, 1), lookup[j]) 
#         y_data.append(y_values)

    else:
        continue
        
image_rgb = np.array(image_rgb, dtype = 'float32')
# x_data = np.array(x_data, dtype=np.uint8)
image_rgb = image_rgb.reshape((len(image_rgb), 224, 224, 3))
image_rgb /= 255

y_data = np.array(y_data)
y_data = to_categorical(y_data)

#### Silhouettes data

In [83]:
X_data = []
y_data = []
image_path = []
gesture = []
image_rgb = []

# directory_in_str = '/home/ubuntu/project_kojak/frames/silhouettes'
directory = os.fsencode('/home/ubuntu/project_kojak/frames/silhouettes')

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".jpg"): 
        path = os.path.join('/home/ubuntu/project_kojak/frames/silhouettes', filename)
        gesture_name = gestures[filename[:2]]
        
        gesture.append(gesture_name)
        y_data.append(gestures_map[gesture_name])
        image_path.append(path)

        
        img = Image.open(path).convert('L')
        img = img.resize((224, 224))
        arr = np.array(img)
        X_data.append(arr)
        
        img2rgb = image_utils.load_img(path=path, target_size=(224, 224))
        img2rgb = image_utils.img_to_array(img2rgb)
        image_rgb.append(img2rgb)

        


#         y_values = np.full((count, 1), lookup[j]) 
#         y_data.append(y_values)

    else:
        continue
        
image_rgb = np.array(image_rgb, dtype = 'float32')
# x_data = np.array(x_data, dtype=np.uint8)
image_rgb = image_rgb.reshape((len(image_rgb), 224, 224, 3))
image_rgb /= 255

y_data = np.array(y_data)
y_data = to_categorical(y_data)

In [74]:
def process_data(x_data, y_data):
    x_data = np.array(x_data, dtype = 'float32')
    # x_data = np.array(x_data, dtype=np.uint8)
    x_data = x_data.reshape((len(x_data), 224, 224, 1))
    x_data /= 255
    
    y_data = np.array(y_data)
    y_data = y_data.reshape(len(x_data), 1)
    y_data = to_categorical(y_data)
    return x_data, y_data

def process_data_rgb(x_data, y_data):
    x_data = np.array(x_data, dtype = 'float32')
    # x_data = np.array(x_data, dtype=np.uint8)
    x_data = x_data.reshape((len(x_data), 224, 224, 3))
    x_data /= 255
    
    y_data = np.array(y_data)
    y_data = to_categorical(y_data)
    return x_data, y_data

In [119]:
# Send the dictionaries to a dataframe to be saved for future use
d = {'image_path':image_path, 'gesture':gesture, 'image_rgb': image_rgb, 'image_bw_x': X_data, 'image_bw_y': y_data}
df = pd.DataFrame(d)
# df['gesture_num'] = df['gesture'].apply(lambda x: x[1:2])
# df['gesture_name'] = df['gesture'].apply(lambda x: x[3:])

# df.to_csv('silhouette_df.csv')
df = pd.read_csv('silhouette_df.csv')

In [23]:
# X_data, y_data = process_data(X_data, y_data)
# image_rgb, y_data = process_data_rgb(image_rgb, y_data)

#### Train-test split

In [103]:
X_train_rgb, X_test_rgb, y_train_rgb, y_test_rgb = train_test_split(image_rgb, y_data, test_size = 0.2, random_state=12)

In [34]:
X_train_rgb, y_train_rgb = process_data_rgb(X_train_rgb, y_train_rgb)
X_test_rgb, y_test_rgb = process_data_rgb(X_test_rgb, y_test_rgb)

In [40]:
# load VGG16
#Get back the convolutional part of a VGG network trained on ImageNet
import keras
from keras import models, layers, optimizers
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.models import Model
imageSize=224
model1 = VGG16(weights='imagenet', include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = optimizers.Adam()

datagen = ImageDataGenerator(
featurewise_center=True,
featurewise_std_normalization=True,
rotation_range=45.,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True)

base_model = model1 # Topless
# Add top layer
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dense(128, activation='relu', name='fc3')(x)
x = Dropout(0.5)(x)
#### Playing with other architectures
#     x = Flatten() (x)
#     x = Dense(64) (x)
#     x = Activation('relu') (x)
#     x = Dropout(0.5) (x)
#     x = Dense(32) (x)
#     x = Activation('relu') (x)
#     x = Dropout(0.5) (x)
####
predictions = Dense(6, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Train top layer
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
      optimizer=optimizers.Adam(), 
      metrics=['accuracy'])
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]

datagen.fit(X_train_rgb)

# Fit model
model.fit_generator(datagen.flow(X_train_rgb, y_train_rgb, batch_size=64),
                steps_per_epoch=len(X_train_rgb) / 64, epochs=8, validation_data=(X_test_rgb, y_test_rgb))
#class_weight=classweight,

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/preprocessing/image.py:440: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  image.ImageDataGenerator.__init__).args:
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/utils/data_utils.py:651: DeprecationWarning: `wait_time` is not used anymore.
  DeprecationWarning)


Epoch 1/8
19/18 [==============================] - 13s 697ms/step - loss: 1.7208 - acc: 0.2992 - val_loss: 1.1924 - val_acc: 0.6128
Epoch 2/8
19/18 [==============================] - 12s 652ms/step - loss: 1.0182 - acc: 0.6347 - val_loss: 0.5280 - val_acc: 0.8485
Epoch 3/8
19/18 [==============================] - 12s 657ms/step - loss: 0.6081 - acc: 0.7697 - val_loss: 0.4251 - val_acc: 0.8451
Epoch 4/8
19/18 [==============================] - 13s 658ms/step - loss: 0.3924 - acc: 0.8663 - val_loss: 0.2187 - val_acc: 0.9226
Epoch 5/8
19/18 [==============================] - 13s 659ms/step - loss: 0.3418 - acc: 0.8867 - val_loss: 0.1596 - val_acc: 0.9529
Epoch 6/8
19/18 [==============================] - 12s 655ms/step - loss: 0.2408 - acc: 0.9238 - val_loss: 0.2789 - val_acc: 0.9091
Epoch 7/8
19/18 [==============================] - 13s 660ms/step - loss: 0.2518 - acc: 0.9147 - val_loss: 0.3006 - val_acc: 0.8990
Epoch 8/8
19/18 [==============================] - 13s 658ms/step - loss: 0.

#### Save and import model

In [87]:
!pwd

/home/ubuntu/project_kojak


In [88]:
# model.save('silhouette_VGG.h5')

from keras.models import load_model
model = load_model('/home/ubuntu/project_kojak/silhouette_VGG.h5')

In [94]:
X_test_rgb[0].shape

(224, 224, 3)

In [92]:
model.predict(X_test_rgb)

array([[2.5975669e-03, 6.8540269e-01, 8.0532441e-03, 4.6215006e-03,
        2.7723959e-01, 2.2085395e-02],
       [1.8037352e-04, 9.8982668e-01, 2.2430022e-04, 8.4401938e-05,
        9.5383069e-03, 1.4591584e-04],
       [2.7303877e-05, 6.1461215e-05, 4.7820024e-04, 7.3979894e-04,
        4.0442785e-04, 9.9828881e-01],
       ...,
       [1.7109163e-04, 3.7865416e-04, 2.1471193e-03, 2.1336788e-02,
        1.3310532e-02, 9.6265584e-01],
       [4.4995941e-05, 1.9672023e-02, 3.9480557e-04, 1.5412706e-04,
        9.7355169e-01, 6.1823567e-03],
       [2.1220765e-05, 2.9467508e-05, 9.6603344e-06, 9.9880779e-01,
        1.1285117e-03, 3.3434828e-06]], dtype=float32)

In [52]:
pred = model.predict(X_test_rgb)
pred = np.argmax(pred, axis=1)
y_true = np.argmax(y_test_rgb, axis=1)

In [97]:
x = X_test_rgb[0]

In [99]:
x = x.reshape(1,224,224,3)

In [100]:
model.predict(x)

array([[0.00259756, 0.68540215, 0.00805323, 0.00462149, 0.27724028,
        0.02208538]], dtype=float32)

In [53]:
print(confusion_matrix(y_true, pred))
print('\n')
print(classification_report(y_true, pred))

[[40  1  0  0  0  0]
 [ 0 39  0  0  3  1]
 [ 0  0 49  1  0  0]
 [ 0  1  0 52  0  0]
 [ 0  1  0  0 48  0]
 [ 0  0  1  1  3 56]]


             precision    recall  f1-score   support

          0       1.00      0.98      0.99        41
          1       0.93      0.91      0.92        43
          2       0.98      0.98      0.98        50
          3       0.96      0.98      0.97        53
          4       0.89      0.98      0.93        49
          5       0.98      0.92      0.95        61

avg / total       0.96      0.96      0.96       297



In [50]:
y_test_rgb

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [55]:
pred = np.argmax(model.predict(X_test_rgb), axis=1)

In [63]:
gesture_names = {0: 'C',
                 1: 'Fist',
                 2: 'L',
                 3: 'Okay',
                 4: 'Palm',
                 5: 'Peace'}

def predict_rgb_image_vgg(path):
    img2rgb = image_utils.load_img(path=path, target_size=(224, 224))
    img2rgb = image_utils.img_to_array(img2rgb)
#     image_rgb.append(img2rgb)
    img2rgb = img2rgb.reshape(1, 224, 224, 3)
    return gesture_names[np.argmax(model.predict(img2rgb))]

In [ ]:
predict_rgb_image_vgg('images_to_predict/test - palm.jpg')

#### Image Augmentation

In [104]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=45.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_rgb)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_train_rgb, y_train_rgb, batch_size=128),
                    steps_per_epoch=len(X_train_rgb) / 128, epochs=10, validation_data=(X_test_rgb, y_test_rgb))

Epoch 1/10
8/7 [================================] - 11s 1s/step - loss: 1.6103 - acc: 0.5610 - val_loss: 1.5952 - val_acc: 0.3856
Epoch 2/10
8/7 [================================] - 9s 1s/step - loss: 0.8910 - acc: 0.6878 - val_loss: 1.5901 - val_acc: 0.4364
Epoch 3/10
8/7 [================================] - 10s 1s/step - loss: 0.6128 - acc: 0.7967 - val_loss: 1.5757 - val_acc: 0.4492
Epoch 4/10
8/7 [================================] - 10s 1s/step - loss: 0.5136 - acc: 0.8229 - val_loss: 1.1850 - val_acc: 0.5508
Epoch 5/10
8/7 [================================] - 10s 1s/step - loss: 0.4274 - acc: 0.8570 - val_loss: 1.3173 - val_acc: 0.5424
Epoch 6/10
8/7 [================================] - 10s 1s/step - loss: 0.3949 - acc: 0.8673 - val_loss: 1.0941 - val_acc: 0.6017
Epoch 7/10
8/7 [================================] - 10s 1s/step - loss: 0.3201 - acc: 0.9048 - val_loss: 0.9776 - val_acc: 0.6483
Epoch 8/10
8/7 [================================] - 10s 1s/step - loss: 0.3264 - acc: 0.890

In [105]:
model.save('drawing_VGG.h5')

In [109]:
np.argmax(model.predict(X_test_rgb[0].reshape(1,224,224,3)))

4